In [56]:
import sys
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, roc_auc_score, roc_curve
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_validate
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

# Load data 

In [57]:
# Read the training dataset from 'train_df.csv' into a DataFrame
train_df = pd.read_csv('train_df.csv')

# Read the test dataset from 'test_df.csv' into a DataFrame
test_df = pd.read_csv('test_df.csv')

# Read the validation dataset from 'val_df.csv' into a DataFrame
val_df = pd.read_csv('val_df.csv')

In [58]:
#Selected Features (Top 10):
important_column_10= [
    'Age Group', 'Num of Bad Mental Health Days', 'Hours of Sleeping', 
    'Arthritis', 'Days Drinking', 'Pneumonia Vaccine', 'Tetanus Last 10 Years', 
    'Income', 'BMI', 'Ethnicity_White', 'Cancer'
    ]
# Select the important columns for RFE from the training, testing, and validation DataFrames
train_df_10 = train_df[important_column_10]
test_df_10=  test_df[important_column_10]
val_df_10 = val_df[important_column_10]

# Separate the features and target variable from the training dataset
X_train_10= train_df_10.drop('Cancer', axis=1)
y_train_10 = train_df_10['Cancer']

# Separate the features and target variable from the testing dataset
X_test_10= test_df_10.drop('Cancer', axis=1)
y_test_10 = test_df_10['Cancer']

In [59]:
#Selected Features (Top 20):
important_column_20 = [
    'Gender', 'Age Group', 'Num of Bad Mental Health Days', 'Years Since Last Checkup', 
    'Hours of Sleeping', 'Arthritis', 'Married', 'Deaf', 'Age Started Smoking', 
    'Cigarettes per Day', 'Days Drinking', 'Flu Vaccine', 'Pneumonia Vaccine', 
    'Tetanus Last 10 Years', 'Had COVID', 'Metropolitan', 'Income', 'Insulin', 
    'BMI', 'Ethnicity_White', 'Cancer'
]
# Select the important columns for RFE from the training, testing, and validation DataFrames
train_df_20 = train_df[important_column_20]
test_df_20=  test_df[important_column_20]
val_df_20 = val_df[important_column_20]

# Separate the features and target variable from the training dataset
X_train_20= train_df_20.drop('Cancer', axis=1)
y_train_20 = train_df_20['Cancer']

# Separate the features and target variable from the testing dataset
X_test_20= test_df_20.drop('Cancer', axis=1)
y_test_20 = test_df_20['Cancer']

In [60]:
#Selected Features (Top 30):
important_column_30 = [
    'Gender', 'Age Group', 'Num of Bad Mental Health Days',
    'Years Since Last Checkup', 'Exercise in Past 30 Days',
    'Hours of Sleeping', 'Heart Disease', 'Asthma', 'Depression',
    'Kidney Disease', 'Arthritis', 'Diabetes', 'Married', 'Deaf', 'Blind',
    'Age Started Smoking', 'Cigarettes per Day', 'Days Drinking',
    'Flu Vaccine', 'Pneumonia Vaccine', 'Tetanus Last 10 Years',
    'Had COVID', 'Metropolitan', 'Income', 'Insulin', 'BMI',
    'Ethnicity_Hispanic', 'Ethnicity_White', 'Education_attended_college',
    'Education_graduated_college', 'Cancer'
]

# Select the important columns for RFE from the training, testing, and validation DataFrames
train_df_30 = train_df[important_column_30]
test_df_30=  test_df[important_column_30]
val_df_30 = val_df[important_column_30]

# Separate the features and target variable from the training dataset
X_train_30= train_df_30.drop('Cancer', axis=1)
y_train_30 = train_df_30['Cancer']

# Separate the features and target variable from the testing dataset
X_test_30= test_df_30.drop('Cancer', axis=1)
y_test_30 = test_df_30['Cancer']

In [61]:
# Separate the features and target variable from the training dataset
X_train_41= train_df.drop('Cancer', axis=1)
y_train_41 = train_df['Cancer']

# Separate the features and target variable from the testing dataset
X_test_41= test_df.drop('Cancer', axis=1)
y_test_41 = test_df['Cancer']

Functions to create Gaussian Naive Bayes, Support Vector Machine, and Random Forest models with specified parameters.


In [62]:
# Function to create and return a Gaussian Naive Bayes model with specified var_smoothing parameter
def nb_model():
    model = GaussianNB(var_smoothing=3.125)
    return model

# Function to create and return a Support Vector Machine model using SGDClassifier with specified parameters
def svm_model():
    model = SGDClassifier(
        loss='hinge',            # Hinge loss for SVM
        penalty='l2',            # L2 regularization
        learning_rate='adaptive', # Adaptive learning rate
        alpha=0.5,               # Regularization term
        eta0=0.025,              # Initial learning rate
        random_state=42          # Seed for reproducibility
    )
    return model

# Function to create and return a Random Forest model with specified parameters
def rf_model():
    model = RandomForestClassifier(
        n_estimators=30,         # Number of trees in the forest
        criterion='gini',        # Function to measure the quality of a split
        max_depth=3,             # Maximum depth of the tree
        random_state=42          # Seed for reproducibility
    )
    return model


Function to perform cross-validation with SMOTE on a given model, returning average accuracy, recall, precision, and F1 scores.


In [63]:
def cross_validation_scores(model, X, y, cv_folds=5):
    """
    Perform cross-validation on the given model using SMOTE to handle class imbalance and return the average scores.

    Parameters:
    - model: The machine learning model to evaluate.
    - X: The feature set.
    - y: The target variable.
    - cv_folds: The number of cross-validation folds (default is 5).

    Returns:
    A dictionary containing the average accuracy, recall, precision, and F1 scores from the cross-validation.
    """
    smote = SMOTE(random_state=42)

    # Apply SMOTE to the training data to handle class imbalance
    X, y = smote.fit_resample(X, y)

    # Define the scoring metrics for cross-validation
    scoring = ['recall_macro', 'accuracy', 'precision_macro', 'f1_macro']

    # Perform cross-validation and obtain the scores
    scores = cross_validate(model, X, y, cv=cv_folds, scoring=scoring)
    
    # Calculate and return the average scores
    return print(f'accuracy: {np.mean(scores["test_accuracy"]):.2f}\n'
             f'recall: {np.mean(scores["test_recall_macro"]):.2f}\n'
             f'precision: {np.mean(scores["test_precision_macro"]):.2f}\n'
             f'f1: {np.mean(scores["test_f1_macro"]):.2f}')


Functions to perform cross-validation using Gaussian Naive Bayes, Support Vector Machine, and Random Forest models, returning average evaluation metrics.


In [64]:
def cross_validate_naive_bayes(X, y, cv_folds=5):
    """
    Perform cross-validation using a Gaussian Naive Bayes model.
    
    Parameters:
    - X: The feature set.
    - y: The target variable.
    - cv_folds: The number of cross-validation folds (default is 3).
    
    Returns:
    A dictionary containing the average accuracy, recall, precision, and F1 scores.
    """
    model = nb_model() 
    return cross_validation_scores(model, X, y, cv_folds)

def cross_validate_svm(X, y, cv_folds=5):
    """
    Perform cross-validation using a Support Vector Machine (SGDClassifier) model.
    
    Parameters:
    - X: The feature set.
    - y: The target variable.
    - cv_folds: The number of cross-validation folds (default is 5).
    
    Returns:
    A dictionary containing the average accuracy, recall, precision, and F1 scores.
    """
    sgdc = svm_model() 
    return cross_validation_scores(sgdc, X, y, cv_folds)

def cross_validate_random_forest(X, y, cv_folds=5):
    """
    Perform cross-validation using a Random Forest model.
    
    Parameters:
    - X: The feature set.
    - y: The target variable.
    - cv_folds: The number of cross-validation folds (default is 5).
    
    Returns:
    A dictionary containing the average accuracy, recall, precision, and F1 scores.
    """
    model = rf_model()
    return cross_validation_scores(model, X, y, cv_folds)


Function to evaluate a machine learning model using SMOTE, calculating and returning accuracy, recall, precision, and F1 scores for both training and testing sets.

In [65]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    """
    Evaluate the performance of a Naive Bayes model using SMOTE to handle class imbalance.

    Parameters:
    - model: The Naive Bayes model to evaluate.
    - X_train: The training feature set.
    - y_train: The training target variable.
    - X_test: The testing feature set.
    - y_test: The testing target variable.

    Returns:
    A string representation of the model's performance metrics: accuracy, recall, precision, and F1 score for both training and testing sets.
    """
    # Initialize SMOTE with a random state for reproducibility
    smote = SMOTE(random_state=42)

    # Apply SMOTE to the training data to handle class imbalance
    X_train, y_train = smote.fit_resample(X_train, y_train)
    
    # Fit the model on the resampled training data
    model.fit(X_train, y_train)
    
    # Predict the target variable for the training set
    y_train_pred = model.predict(X_train)
    
    # Predict the target variable for the test set
    y_test_pred = model.predict(X_test)
    
    # Calculate evaluation metrics for the training set
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)
    
    # Calculate evaluation metrics for the test set
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    
    # Print and return the evaluation metrics
    model_scores = print(
        f'Training Scores: \n'
        f'Accuracy: {train_accuracy:.2f} \n'
        f'Recall: {train_recall:.2f}\n'
        f'Precision: {train_precision:.2f}\n'
        f'F1: {train_f1:.2f}\n\n'
        f'Test Scores:\n'
        f'Accuracy: {test_accuracy:.2f}\n'
        f'Recall: {test_recall:.2f}\n'
        f'Precision: {test_precision:.2f}\n'
        f'F1: {test_f1:.2f}\n'
    )
    #print(model_scores)
    return model_scores


# Evaluating Models with Different Numbers of Selected Features
This evaluates the performance of three machine learning models (Naive Bayes, Random Forest, and Support Vector Machine) on datasets with different numbers of top selected features (10, 20, 30, and 41). For each set of selected features, the script prints the train and test scores (accuracy, recall, precision, and F1 score) for each model. The models are evaluated using a custom `evaluate_model` function that incorporates SMOTE to handle class imbalance.

In [69]:
# Initialize the Naive Bayes, Random Forest, and Support Vector Machine models
model_nb = nb_model()
model_rf = rf_model()
model_svm = svm_model()

In [83]:
# Evaluate models with the top 10 selected features
print('Top 10 Selected Features ')

print('\nNB performance\n')
model_nb = nb_model()
evaluate_model(model_nb, X_train_10, y_train_10, X_test_10, y_test_10)

print('\nRF performance\n')
model_rf = rf_model()
evaluate_model(model_rf, X_train_10, y_train_10, X_test_10, y_test_10)

print('\nSMV performance\n')
model_svm = svm_model()
evaluate_model(model_svm, X_train_10, y_train_10, X_test_10, y_test_10)

Top 10 Selected Features 

NB performance

Training Scores: 
Accuracy: 0.69 
Recall: 0.83
Precision: 0.65
F1: 0.73

Test Scores:
Accuracy: 0.58
Recall: 0.83
Precision: 0.19
F1: 0.31


RF performance

Training Scores: 
Accuracy: 0.71 
Recall: 0.82
Precision: 0.67
F1: 0.74

Test Scores:
Accuracy: 0.59
Recall: 0.81
Precision: 0.19
F1: 0.31


SMV performance

Training Scores: 
Accuracy: 0.69 
Recall: 0.85
Precision: 0.64
F1: 0.73

Test Scores:
Accuracy: 0.57
Recall: 0.84
Precision: 0.19
F1: 0.31



In [84]:
# Evaluate models with the top 20 selected features
print('Top 20 Selected Features ')

print('\nNB performance\n')
evaluate_model(model_nb, X_train_20, y_train_20, X_test_20, y_test_20)

print('\nRF performance\n')
evaluate_model(model_rf, X_train_20, y_train_20, X_test_20, y_test_20)

print('\nSMV performance\n')
evaluate_model(model_svm, X_train_20, y_train_20, X_test_20, y_test_20)

Top 20 Selected Features 

NB performance

Training Scores: 
Accuracy: 0.69 
Recall: 0.87
Precision: 0.64
F1: 0.73

Test Scores:
Accuracy: 0.54
Recall: 0.86
Precision: 0.18
F1: 0.30


RF performance

Training Scores: 
Accuracy: 0.72 
Recall: 0.77
Precision: 0.70
F1: 0.74

Test Scores:
Accuracy: 0.67
Recall: 0.70
Precision: 0.21
F1: 0.32


SMV performance

Training Scores: 
Accuracy: 0.70 
Recall: 0.85
Precision: 0.65
F1: 0.74

Test Scores:
Accuracy: 0.58
Recall: 0.84
Precision: 0.19
F1: 0.31



In [85]:
# Evaluate models with the top 30 selected features
print('Top 30 Selected Features ')
print('\nNB performance\n')
evaluate_model(model_nb, X_train_30, y_train_30, X_test_30, y_test_30)

print('\nRF performance\n')
evaluate_model(model_rf, X_train_30, y_train_30, X_test_30, y_test_30)

print('\nSMV performance\n')
evaluate_model(model_svm, X_train_30, y_train_30, X_test_30, y_test_30)

Top 30 Selected Features 

NB performance

Training Scores: 
Accuracy: 0.68 
Recall: 0.88
Precision: 0.63
F1: 0.74

Test Scores:
Accuracy: 0.53
Recall: 0.87
Precision: 0.18
F1: 0.30


RF performance

Training Scores: 
Accuracy: 0.74 
Recall: 0.81
Precision: 0.70
F1: 0.75

Test Scores:
Accuracy: 0.65
Recall: 0.73
Precision: 0.21
F1: 0.32


SMV performance

Training Scores: 
Accuracy: 0.70 
Recall: 0.85
Precision: 0.65
F1: 0.74

Test Scores:
Accuracy: 0.58
Recall: 0.84
Precision: 0.19
F1: 0.31



In [86]:
# Evaluate models with the top 41 selected features
print('Top 41 Selected Features')
print('\nNB performance\n')
evaluate_model(model_nb, X_train_41, y_train_41, X_test_41, y_test_41)

print('\nRF performance')
evaluate_model(model_rf, X_train_41, y_train_41, X_test_41, y_test_41)

print('\nSMV performance')
evaluate_model(model_svm, X_train_41, y_train_41, X_test_41, y_test_41)

Top 41 Selected Features

NB performance

Training Scores: 
Accuracy: 0.68 
Recall: 0.90
Precision: 0.62
F1: 0.74

Test Scores:
Accuracy: 0.51
Recall: 0.89
Precision: 0.17
F1: 0.29


RF performance
Training Scores: 
Accuracy: 0.74 
Recall: 0.76
Precision: 0.72
F1: 0.74

Test Scores:
Accuracy: 0.69
Recall: 0.65
Precision: 0.22
F1: 0.33


SMV performance
Training Scores: 
Accuracy: 0.70 
Recall: 0.85
Precision: 0.65
F1: 0.74

Test Scores:
Accuracy: 0.58
Recall: 0.84
Precision: 0.19
F1: 0.31



In [74]:
print('Top 10 Selected Features ')

print('\nNB cross validation scores')
cross_validate_naive_bayes(X_train_10, y_train_10)

print('\nSVM cross validation scores')
cross_validate_svm(X_train_10, y_train_10)

print('\nRF cross validation scores')
cross_validate_random_forest(X_train_10, y_train_10)

Top 10 Selected Features 

NB cross validation scores
accuracy: 0.69
recall: 0.69
precision: 0.71
f1: 0.68

SVM cross validation scores
accuracy: 0.69
recall: 0.69
precision: 0.71
f1: 0.68

RF cross validation scores
accuracy: 0.71
recall: 0.71
precision: 0.72
f1: 0.71


In [75]:
print('Top 20 Selected Features ')

print('\nNB cross validation scores')
cross_validate_naive_bayes(X_train_20, y_train_20)

print('\nSVM cross validation scores')
cross_validate_svm(X_train_20, y_train_20)

print('\nRF cross validation scores')
cross_validate_random_forest(X_train_20, y_train_20)

Top 20 Selected Features 

NB cross validation scores
accuracy: 0.69
recall: 0.69
precision: 0.71
f1: 0.68

SVM cross validation scores
accuracy: 0.70
recall: 0.70
precision: 0.72
f1: 0.69

RF cross validation scores
accuracy: 0.72
recall: 0.72
precision: 0.73
f1: 0.72


In [76]:
print('Top 30 Selected Features ')

print('\nNB cross validation scores')
cross_validate_naive_bayes(X_train_30, y_train_30)

print('\nSVM cross validation scores')
cross_validate_svm(X_train_30, y_train_30)

print('\nRF cross validation scores')
cross_validate_random_forest(X_train_30, y_train_30)

Top 30 Selected Features 

NB cross validation scores
accuracy: 0.68
recall: 0.68
precision: 0.72
f1: 0.67

SVM cross validation scores
accuracy: 0.70
recall: 0.70
precision: 0.72
f1: 0.69

RF cross validation scores
accuracy: 0.73
recall: 0.73
precision: 0.74
f1: 0.73


In [77]:
print('Top 41 Selected Features ')

print('\nNB cross validation scores')
cross_validate_naive_bayes(X_train_41, y_train_41)

print('\nSVM cross validation scores')
cross_validate_svm(X_train_41, y_train_41)

print('\nRF cross validation scores')
cross_validate_random_forest(X_train_41, y_train_41)

Top 41 Selected Features 

NB cross validation scores
accuracy: 0.68
recall: 0.68
precision: 0.72
f1: 0.66

SVM cross validation scores
accuracy: 0.70
recall: 0.70
precision: 0.72
f1: 0.69

RF cross validation scores
accuracy: 0.73
recall: 0.73
precision: 0.73
f1: 0.73
